In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import trapz
import os

# READ THREE MAIN FILES FOR REQUIRED DATA FOR COST CALCULATION
# curr_dir = os.path.abspath("")
# df1 = pd.read_csv(os.path.join(curr_dir, "BASE_CASE", "time_series_60min_singleindex.csv"))
df1 = pd.read_csv("time_series_60min_singleindex.csv")
df1['utc_timestamp'] = pd.to_datetime(df1['utc_timestamp'], utc=True)
df_P= pd.read_csv("P_w_100.csv")
df_P['time'] = pd.to_datetime(df_P['time'], utc=True) 
df_P_PV= pd.read_csv("P_PV.csv")
df_P_PV['time'] = pd.to_datetime(df_P['time'], utc=True) 
df_SP_Load= pd.read_excel('Demand_Renewable_Share_Loc_6.xlsx', sheet_name="NL_2017")
df_SP_Load["Date (GMT+1)"] = pd.to_datetime(df_SP_Load["Date (GMT+1)"])
#GMT+1 to UTC
df_SP_Load["Date (UTC)"] = df_SP_Load["Date (GMT+1)"] - pd.Timedelta(hours=1)
# df_SP= df_SP_Load['Day Ahead Auction (DK2)']
df_SP= df_SP_Load['Day Ahead Auction']
# df_Load= df_SP_Load['Load']

# FUNCTIONS
def filter_on_year(df: pd.DataFrame, year: int):
    """This functions filters a dataframe on a year using the utc_timestamp column.

    Args:
        year (int): year e.g. 2017
    Return:
        df: a dataframe filter on the year parameter
    """
    return df[df['time'].dt.year == year]

sel_year= 2017
df_sel_year= filter_on_year(df=df_P, year=sel_year)


# add functions from previous py files to automatize
def normalized_vre(vre):
    """normalizes the VRE generation with max"""
    norm_VRE = vre / np.max(vre)
    # vre? array with VRE gen
    return norm_VRE

def normalized_spot(df_spot: pd.DataFrame):
    """normalizes with mean"""
    sp=df_SP.drop(0)
    norm_SP=sp/sp.mean()
    
    return norm_SP

In [2]:
# GERMANY BFH_LW, 2017 (for gen and SP), starting operation in 2025 (costs references)
# norm_SP_DE_2017= normalized_spot(df_SP)

SP_DE_2017=df_SP_Load['Day Ahead Auction'][1:].astype(float)
# SP_DE_2017=df_SP_Load['Day Ahead Auction (DK2)'][1:].astype(float)

norm_SP_DE_2017=SP_DE_2017/SP_DE_2017.mean()
SP_mean_DE_2017=SP_DE_2017.mean()
p_DE_2017= SP_DE_2017/SP_mean_DE_2017
# p_DE_2017= SP_DE_2017/norm_SP_DE_2017

# AEP= sum(np.array(df_sel_year['Off_NL_HW'])*50)
AEP= sum(np.array(df_sel_year['LOC_1'])*50)
h_AEP= np.array(df_sel_year['LOC_1']*50)

# AEP= sum(np.array(df_sel_year['BFH_LW'])*50)
# h_AEP= np.array(df_sel_year['BFH_LW']*50)

Gen_COVE= sum(h_AEP*p_DE_2017)

# 2021 paper: most recent paper: use it as base for future. Assumption of increase or decrease for 2025, 2035 adn 2045 as in paper (by 68,6%)
# from Table A.3 Appendix A, the avg capex in MEur/MW for years 2025, 2035, 2045:
on_CAPEX_fut = [1855000, 1675000, 1540000] # Eur/MW - I have the total MW per year
lifetime= 25 
# ann_on_CAPEX_fut = on_CAPEX_fut/lifetime
ann_on_CAPEX_fut = [capex / lifetime for capex in on_CAPEX_fut]
off_CAPEX_fut = [2, 1.8, 1.8]
# for extreme scenarios i will use the min and max values of CaPex provided
on_VarOPEX = [1.47, 1.40, 1.22] # Eur/MWh - full load hours?
on_FixOPEX= [13720, 12350, 11360] # Eur/MW/yr - I have the total MW per year
operating_cost = np.array(on_FixOPEX)*50  # EUR/year
capital_cost = (np.array(on_CAPEX_fut)*50) /25   # EUR
costs= operating_cost[0] + capital_cost[0]


LCOE_BFH_DE_2017= costs/ AEP

COVE_BFH_DE_2017= costs/(Gen_COVE)
VF=LCOE_BFH_DE_2017/COVE_BFH_DE_2017
print(LCOE_BFH_DE_2017, COVE_BFH_DE_2017, VF)

38.03365978891316 38.40046914834925 0.9904477896345738


In [30]:
# for BFH_LW: higher COVE than LCOE. 
# Result DE_2017: 38.03365978891316 44.97887217374359
# Result_DK2_2017: 24.666236391592143 27.935247000752387
# Result_Off_NL_HW: 21.730405110164785 22.525547777710297

Gen_COVE= sum(h_AEP*p_DE_2017)
Gen_COVE

In [3]:
# COST CALCULATION
# inputs:
capex = 92750000
opex = 13720
years = 30
WACC = 0.06
OM_rate= 0.02

# procedure:
year_frac = np.array(range(years + 1))
disc_factor = [1 / ((1 + WACC) ** year_frac[i]) for i in range(years + 1)]  

PVC = [capex, 0, opex]  
OM_cost=opex*(1+OM_rate)

for i in range(3, years):
    pvc_value = OM_cost * disc_factor[i]  
    PVC.append(pvc_value)

NPV = sum(PVC)  

# AEP
AEP_array= np.zeros(31)
AEP_array[2:] = AEP
PVEC= AEP_array*disc_factor
NPVE= sum(PVEC)

# LCOE
LCOE= NPV/NPVE

# AEP_COVE
p_ratio= p_DE_2017
AEP_COVE= sum(h_AEP*p_ratio)

AEP_COVE_array= np.zeros(31)
AEP_COVE_array[2:] = AEP_COVE
COVE_PVEC= AEP_COVE_array*disc_factor
COVE_NPVE= sum(COVE_PVEC)

# COVE
COVE=NPV/COVE_NPVE

# outputs:
print("LCOE:", LCOE)
print("COVE:", COVE)

LCOE: 62.70780389197066
COVE: 63.312578965022226


WE MODEL p_ratio FOR FUTURE SCENARIOS FOR EACH LOCATION

In [63]:
# p_ratio is defined as hourly spot price by the avg annual spot price. 
# the model will consist on a second order polynomial regression relationship between the hourly action electricity prices and the hourly residual demand (for VREs).
# Key assumptions for this model, as described in the research paper Cost of Valued ENergy for design of renewable energy systems, the main grid service that will be 
# used to make revenues will be the direct participation in the electricity market, and not others like capacity payments, etc. 
# also, FIT have stopped existing since 2019, therefore it will not be considered as a possible scenario option for future scenarios. 

df_SPL = pd.read_excel("Demand_Renewable_Share_Loc_6.xlsx", sheet_name="DE_2017", usecols=["Date (GMT+1)", "Day Ahead Auction", "Load"])
df_SPL["Date (GMT+1)"] = pd.to_datetime(df_SPL["Date (GMT+1)"])
# convert from GMT+1 to UTC by subtracting one hour
df_SPL["Date (UTC)"] = df_SPL["Date (GMT+1)"] - pd.Timedelta(hours=1)
df_SPL.drop("Date (GMT+1)", axis=1, inplace=True)

df = pd.read_csv("time_series_60min_singleindex.csv")
df['utc_timestamp'] = pd.to_datetime(df['utc_timestamp'], utc=True)

df_Load= df_SPL['Load'].drop(0)
max_load= np.array(df_Load).max()
Load_DE_2017= np.array(df_Load)/max_load

df_Spot= df_SPL['Day Ahead Auction'].drop(0)
avg_spot= np.array(df_Spot).mean()
Spot_DE_2017= np.array(df_Spot)/avg_spot

def filter_on_year(df: pd.DataFrame, year: int):
    """This functions filters a dataframe on a year using the utc_timestamp column.

    Args:
        year (int): year e.g. 2017
    Return:
        df: a dataframe filter on the year parameter
    """
    return df[df['utc_timestamp'].dt.year == year]
def vre_share(df_vre: pd.DataFrame, country: str = "DE", year_vre: int = 2017):
    """it calculates the VRE for a country.

    Args:
        df (pd.DataFrame): df filtered on a year.
        country (str, optional): country code for dataframe column. Defaults to "DE".
        year_vre (int): the year for the vre

    Returns:
        _type_: _description_
    """
    # GERMANTY 2017: base example
    df_vre = filter_on_year(df=df_vre, year=year_vre)
    x=df_vre['utc_timestamp']
    y1 =df_vre[f"{country}_wind_offshore_generation_actual"]
    y2 =df_vre[f"{country}_wind_onshore_generation_actual"]
    y3 =df_vre[f"{country}_solar_generation_actual"]

    # Residual Demand= Demand - VRE_production --- linear proven relationship btw SP and Res_Dem
    VRE_sum = y1+y2+y3
    arr_VRE_sum_avg = np.full_like(VRE_sum, VRE_sum.mean())
    # arr_VRE_sum_avg[17545:26305+1] = VRE_sum_avg
    y4 = arr_VRE_sum_avg
    
    # plt.stackplot(x, y1, y2, y3, labels=['Off', 'On', 'Sol'])
    # plt.title(f'VRE generation in {country} in {year_vre}')
    # plt.xlabel('Time[h]')
    # plt.ylabel('Power [MW]')
    # plt.legend(loc='upper left')
    # plt.show()
    
    # print(x.shape, y4.shape)

    # plt.plot(x, y4, label = "Qavg")
    # # plt.plot(x, VRE_sum/sum(VRE_sum))
    # plt.plot(x, VRE_sum)
    # plt.title(f'Hourly VRE generation/Average Demand in {country} in {year_vre}')
    # plt.xlabel('Time[h]')
    # plt.ylabel('Power [MW]')
    # plt.legend(loc='upper left')
    # plt.show()
    
    Q_p= VRE_sum-arr_VRE_sum_avg
    # print(Q_p)
    # VRE_sum = pd.Series(y1) + pd.Series(y2) + pd.Series(y3)
    
    return VRE_sum #, VRE_sum_avg #if i put the ys, it returns a list

# for year in range(2017,2018):
year = 2017
VRE_sum_de = vre_share(df_vre=df, country= "DE", year_vre=year)

# list1= norm_load_DE_2017.tolist()
# arr1 = np.array(list1)
# list2=VRE_sum_de_norm.tolist()
# arr2 = np.array(list2)
# norm_Res_Load_VRE= arr1 - arr2

17545     -453.652283
17546     -736.652283
17547    -1085.652283
17548     -951.652283
17549     -563.652283
             ...     
26300    13940.347717
26301    14171.347717
26302    14538.347717
26303    14777.347717
26304    15760.347717
Length: 8760, dtype: float64
